# 將資料處理成要分群的feature

#### 存成pkl檔，只要執行一次即可

In [1]:
import pickle
import pandas as pd
import numpy as np
import os
from datetime import datetime,timedelta,date,time
import glob
%matplotlib inline

In [2]:
column_names = ['DateTime', 'ratio_value']
area = 'data/ratiodata/'
dir_path = os.getcwd()

In [3]:
region = pd.read_excel(('region_full.xlsx'), sheetname=None, header=0, skiprows=None)
region_list = ['Yonghe','Xinyi', 'Penghu', 'Dali', 'Banciao','Quchi']

In [5]:
## people with zero value whole year (which caused the problem of "Mean of empty slice")
# 永和
r1 = ['T1411','T0457','T0031','T2834','T0639','T0473','T2915','T1298','T0441','T0603','T0474','T7018',
      'T1267','T1264','T3265','T2879','T0980','T2914','T3072','T3522']
# 信義
r2 = ['M00686']
# 板橋
r3 = ['T4166','T6540','T4648','T4542','T6873','T5003','T4167','T5851']
remove_list = [r1,r2,None,None,r3,None]

In [ ]:
dt = {}
for dpath in glob.glob(os.path.join(dir_path,area)+'*.csv'):
    d_name = dpath[61:][:-4]
    #print(d_name)
    dt[d_name] = pd.read_csv(dpath)
    dt[d_name].fillna(0,inplace = True)
    dt[d_name].columns = column_names
    dt[d_name]['DateTime'] = pd.to_datetime(dt[d_name]['DateTime'])  

In [62]:
# 輸入一個 DataFrame，回傳非假日的所有紀錄
def get_weekday_DT(dataT):
    return dataT[[0<=dtime.weekday()<=4 for dtime in dataT['DateTime']]]
#get_weekday_DT(dt['M00001.csv'])

In [63]:
# 將所有的非假日資料過濾出來
summer_dt = {}
for k in dt.keys():
    summer_dt[k] = get_weekday_DT(dt[k])

In [64]:
# all_times 為所有時間點的 datetime.time，[00:00, 01:0, 02:00, ... , 23:00,00:00]
all_times = [time(hour=1, minute=0, second=0)]
while True:
    next_time = datetime.combine(date.today(), all_times[-1]) + timedelta(hours=1)
    #next_time = all_times[-1] + time.timedelta(minutes=15)
    if next_time.time() == all_times[0]:
        break
    all_times.append(next_time.time())

In [65]:
# 將每個 user 的所有時間點用電情形做整理
# summer_tdt 為 dict, key: 用戶名稱
#                   , value: 該用戶的每個時間點用電情形, 為 dict
# 用戶用電情形, key: 時間點, ex: 01:00
#           , value: list, 所有該用戶在該時間點用電的紀錄, ex: [ 0.2, 0.3, 0.4]
# diff       為 dict, key: 用戶名稱
#                   , value: 該用戶的每個時間點用電斜率, 為 dict
# 用戶用電斜率, key: 時間點, ex: 01:00
#           , value: list, 所有該用戶在該時段與下一個時段的斜率


summer_tdt = {}
diff = {}
for k in summer_dt.keys():
    #print(k)
    summer_tdt[k] = {}
    diff[k] = {}
    #將每用戶每個時間點歸零
    for t in all_times:
        summer_tdt[k][t] = []
        diff[k][t]=[]
    for i in range(len(summer_dt[k])):
        if np.isnan(summer_dt[k]['ratio_value'].iloc[i]):
            continue
        summer_tdt[k][summer_dt[k]['DateTime'].iloc[i].time()].append(summer_dt[k]['ratio_value'].iloc[i])
        if i < len(summer_dt[k])-1:
            diff[k][summer_dt[k]['DateTime'].iloc[i].time()].append(summer_dt[k]['ratio_value'].iloc[i+1]-summer_dt[k]['ratio_value'].iloc[i])

In [ ]:
A = summer_tdt
summer_tdt = open('summer_tdt.pkl','wb')
pickle.dump(A,summer_tdt)
print(summer_tdt)
summer_tdt.close()
print('done! finsh storing feature matirx48')

In [ ]:
# 將所有 user 的所有時間點用電取 mean，並儲存在 summer_avgdt
# summer_avgdt 為 dict, key: 每個時間點
#                     , value: list, 紀錄每個 user 該時段平均用電
# diff_avgdt   為 dict, key: 每個時間點
#                     , value: list, 紀錄每個 user 該時段與下一個時段的平均斜率

summer_avgdt = {}
diff_avgdt = {}
for t in all_times:
    summer_avgdt[t] = []
    diff_avgdt[t] = []
    for k in summer_dt.keys():
        summer_avgdt[t].append(np.nanmean(summer_tdt[k][t]))
        diff_avgdt[t].append(np.nanmean(diff[k][t]))

In [ ]:
def union2(dict1, dict2):
    return dict(dict1.items() + dict2.items())
dicts=[summer_avgdt,diff_avgdt]
dict(i for dct in dicts for i in dct.items())

# 存成pkl檔

### feature_matrix48.pkl 為用電平均與斜率，共48個feature的檔案

In [41]:
#x1 = [summer_avgdt[k] for k in summer_avgdt.keys()]
x1 = summer_avgd
#x2 = [diff_avgdt[k] for k in diff_avgdt.keys()]
x2 = diff_avgdt
x = x1 + x2
#A = [list(i) for i in zip(*x)]
A = x
feature_matrix48 = open('matrix_data48_'+heka+'.pkl','wb')
pickle.dump(A,feature_matrix48)
print(feature_matrix48)
feature_matrix48.close()
print('done! finsh storing feature matirx48')

<_io.BufferedWriter name='matrix_data_Dali.pkl'>
done! finsh storing feature matirx


### feature_matrix24.pkl 為只有用電平均，共24個feature的檔案

In [ ]:
#B = [list(i) for i in zip(*x1)]
B = x1
feature_matrix24 = open('matrix_data24_'+heka+'.pkl','wb')
pickle.dump(B,feature_matrix24)
print(feature_matrix24)
feature_matrix24.close()
print('done! finsh storing feature matirx24')
#B = pickle.load(open('matrix_data_'+heka+'.pkl','rb'))